# Access to ADMIN$ Share
Detects access to $ADMIN share

## Rule Content
```
- title: Access to ADMIN$ Share
  id: 098d7118-55bc-4912-a836-dc6483a8d150
  description: Detects access to $ADMIN share
  tags:
  - attack.lateral_movement
  - attack.t1077
  status: experimental
  author: Florian Roth
  logsource:
    product: windows
    service: security
    definition: The advanced audit policy setting "Object Access > Audit File Share"
      must be configured for Success/Failure
    category: null
  detection:
    selection:
      EventID: 5140
      ShareName: Admin$
    filter:
      SubjectUserName: '*$'
    condition: selection and not filter
  falsepositives:
  - Legitimate administrative activity
  level: low

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-endpoint-winevent-security-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='((event_id:"5140" AND share_name:"Admin$") AND (NOT (user_name.keyword:*$)))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()